# Описание проекта

Предоставлены пробы нефти в трёх регионах. Построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой *Bootstrap*.

### Описание данных:
<ul><li>id — уникальный идентификатор скважины;</li>
    <li>f0, f1, f2 — три признака точек (синтетические важные признаки);</li>
    <li>product — объём запасов в скважине (тыс. баррелей).</li></ul>
    
### Условия задачи:
<ul><li>Модель - линейная регрессия.</li>
<li>Исследуется 500 точек, из которых выбирают 200 лучших.</li>
<li>Бюджет - 10 млрд рублей.</li>
<li>Доход с одного барреля - 450 рублей дохода.</li>
<li>Регион с риском убытков менее 2.5%..</li></ul>

### План работы:
<a href='#step_1'>Шаг 1. Получение общей информации и подготвка данных.</a><br>
На первом этапе импортируем необходимые библиотеки и читаем файл, знакомимся с общей информацией, выставляем необходимые типы данных, проверяем пропуски.<br>
<a href='#step_2'>Шаг 2. Разбиение данных и обучение модели.</a><br>
На втором шаге разбиваем данные на выборки, обучаем модели, проверяем её.<br>
<a href='#step_3'>Шаг 3. Подготовка к расчёту прибыли.</a><br>
Определяем необходимые переменные. Рассчитываем необходимое минимальное значение дохода скважины.<br>
<a href='#step_4'>Шаг 4. Расчёт прибыли.</a><br>
Создаем функцию для расчета прибыли в регионе.<br>
<a href='#step_5'>Шаг 5. Расчёт рисков.</a><br>
Проводим расчет доверительного интервала, рисков.

# 1. Загрузка и подготовка данных

<a id = 'step_1'></a>

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")
from numpy.random import RandomState
from scipy import stats as st

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')
all_data = [data_0, data_1, data_2]

In [3]:
for element in all_data:
    print(element.describe())
    print()
    print(element.info())
    print()
    print()

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


                  f0             f1             f2        produ

Пропусков нет. Типы данных и сами данные корректны, кроме product - переведем сразу в прибыль.<br>
Уберем столбец с id, для задания он не нужен

In [4]:
for element in all_data:
    element.drop('id', axis = 'columns', inplace = True)

Проведем масштабирование данных фич f0-f2

In [5]:
scaler = StandardScaler()
for element in all_data:
    features = element.drop(['product'], axis = 'columns')
    target = element['product']
    
    features_train,features_valid, target_train, target_valid = train_test_split(
        features, target, test_size = 0.25, random_state = 41)
    
    scaler.fit(features_train)
    element[['f0', 'f1', 'f2']] = scaler.transform(features)

In [6]:
data_0, data_1, data_2 = all_data[0], all_data[1], all_data[2]

In [7]:
print(data_0)

             f0        f1        f2     product
0      0.236728 -1.486488 -0.393028  105.280062
1      0.958280 -1.173347  0.574765   73.037750
2      0.600377 -0.195828 -0.331845   85.265647
3     -0.609811 -0.221564  0.147953  168.620776
4      1.708228 -0.189030  0.693799  154.036647
...         ...       ...       ...         ...
99995  0.542127  0.239077  1.101237  110.744026
99996  1.024494 -1.257645 -0.376792  122.346843
99997  0.608238 -0.460397 -1.183992   64.375443
99998  0.572191 -1.547582 -0.281378   74.040764
99999  1.451626 -1.026869  0.992728  149.633246

[100000 rows x 4 columns]


Данные готовы для работы с ними.

# 2. Обучение и проверка модели

<a href='#step_1'>Шаг 1. Получение общей информации и подготвка данных.</a><br>
<a href='#step_2'>Шаг 2. Разбиение данных и обучение модели.</a><br>
<a href='#step_3'>Шаг 3. Подготовка к расчёту прибыли.</a><br>
<a href='#step_4'>Шаг 4. Расчёт прибыли.</a><br>
<a href='#step_5'>Шаг 5. Расчёт рисков.</a><br>

<a id = 'step_2'></a>

In [8]:
all_results = []
all_target_valid = []
for element in all_data:
    features = element.drop(['product'], axis = 'columns')
    target = element['product']
    
    features_train,features_valid, target_train, target_valid = train_test_split(
        features, target, test_size = 0.25, random_state = 41)
    
    all_target_valid.append(target_valid)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    results = model.predict(features_valid)
    all_results.append(pd.Series(results))
    mean_model = sum(results) / len(results)
    rmse = mean_squared_error(target_valid, results) ** 0.5
    print('mean_of_model =', mean_model, 'rmse model =', rmse)
    

mean_of_model = 92.72452841600004 rmse model = 37.6655516764908
mean_of_model = 69.25033902013546 rmse model = 0.8890771171257575
mean_of_model = 94.8043951051491 rmse model = 40.19028851519172


Наибольшее среднее у 1 и 3 региона, но и средняя ошибка у них очень большая. Во втором регионе средняя ошибка в ~4-5 раз меньше, чем в 1 и 3, при этом среднее меньше на 10 млн.<br>
В лучшем исходе 3 регион является самым прибыльным, в худшем - самым убыточным.2 регион самый стабильный.

# 3. Подготовка к расчёту прибыли

<a href='#step_1'>Шаг 1. Получение общей информации и подготвка данных.</a><br>
<a href='#step_2'>Шаг 2. Разбиение данных и обучение модели.</a><br>
<a href='#step_3'>Шаг 3. Подготовка к расчёту прибыли.</a><br>
<a href='#step_4'>Шаг 4. Расчёт прибыли.</a><br>
<a href='#step_5'>Шаг 5. Расчёт рисков.</a><br>

<a id = 'step_3'></a>

In [9]:
COUNTS_POINT = 500
BEST_POINTS = 200
BUDGET = 10000000000
COST = 450000

инициализация основных параметров задачи.

In [10]:
develop_budget = BUDGET / BEST_POINTS / COST

In [11]:
np.ceil(develop_budget)

112.0

Средняя вырука с вышки для безубыточной разработки.<br>
С виду кажется, что добиться такого среднего значения можно в 1 и 3 регионе, если выбирать лучшие скважины.

# 4. Расчёт прибыли

<a href='#step_1'>Шаг 1. Получение общей информации и подготвка данных.</a><br>
<a href='#step_2'>Шаг 2. Разбиение данных и обучение модели.</a><br>
<a href='#step_3'>Шаг 3. Подготовка к расчёту прибыли.</a><br>
<a href='#step_4'>Шаг 4. Расчёт прибыли.</a><br>
<a href='#step_5'>Шаг 5. Расчёт рисков.</a><br>

<a id = 'step_4'></a>

In [12]:
state = RandomState(41)
def profit(count_wells, count_best_wells, number_region):
    possible_wells_indexes = all_results[number_region].sample(n = count_wells, replace = True, random_state = state).index
    possible_wells_best = all_results[number_region][possible_wells_indexes].sort_values(ascending = False)[:count_best_wells].index
    total_revenue = all_target_valid[number_region].reset_index().loc[possible_wells_best, 'product'].sum()
    return total_revenue * COST - BUDGET

функция расчета прибыли региона.

In [13]:
print('доход первого региона =', profit(500, 200, 0))
print('доход второго региона =', profit(500, 200, 1))
print('доход третьего региона =', profit(500, 200, 2))

доход первого региона = 693620736.0506096
доход второго региона = 471279710.92001915
доход третьего региона = 560436383.9433498


проверка функции для каждого региона.

# 5. Расчёт рисков

<a href='#step_1'>Шаг 1. Получение общей информации и подготвка данных.</a><br>
<a href='#step_2'>Шаг 2. Разбиение данных и обучение модели.</a><br>
<a href='#step_3'>Шаг 3. Подготовка к расчёту прибыли.</a><br>
<a href='#step_4'>Шаг 4. Расчёт прибыли.</a><br>
<a href='#step_5'>Шаг 5. Расчёт рисков.</a><br>

<a id = 'step_5'></a>

In [14]:
revenues_0= []
revenues_1= []
revenues_2= []
for i in range(1000):
    revenues_0.append(profit(500,200, 0))
    revenues_1.append(profit(500,200, 1))
    revenues_2.append(profit(500,200, 2))

применение bootstrap для вычисления распределения прибылей регионов.

In [15]:
revenues_0, revenues_1, revenues_2 = pd.Series(revenues_0), pd.Series(revenues_1), pd.Series(revenues_2)
k = 1
for i in [revenues_0, revenues_1, revenues_2]:
    max_quantile = i.quantile(0.975)
    min_quantile = i.quantile(0.025)
    print(str(k)+' регион', 'min =',min_quantile, 'max =', max_quantile)
    k += 1

1 регион min = -99141931.24151698 max = 908834980.5497117
2 регион min = 78088130.65162969 max = 848832698.13874
3 регион min = -113383455.27368487 max = 929484372.5456522


вычисление доверительного интервала.<br>
с большой вероятностью прибыли регионов находятся в этих промежутках.

In [16]:
print((revenues_0 < 0).mean())
print((revenues_1 < 0).mean())
print((revenues_2 < 0).mean())

0.062
0.009
0.06


Вероятность, что прибыль у региона будет отрицательной.<br>
Получилось, что все регионы имеют больше 2.5% шанса убытков, кроме второго.

In [17]:
k = 1
for i in [revenues_0, revenues_1, revenues_2]:
    print(str(k) + ' регион ' + str(i.mean()))
    k += 1

1 регион 392589148.0353256
2 регион 460824976.3647162
3 регион 420877970.2442401


Лучшим оказался 2 регион.

# Общий вывод
При ознакомлении с данными пропусков не выявлено, типы данных и сами данные корректны.<br>
Далее убрал столбец с id - для задания он не нужен. Провел масштабирование данных.<br>
В следующем шаге разбил данные на тестовые и валидационные выборки, обучил модель, вычислил метрику rmse:<ul>
<li>1 регион - 37.5100 продукта.</li>
<li>2 регион - 0.8879 продукта.</li>
<li>3 регион - 40.1470 продукта.</li>
</ul>
После инициализировал основные параметров задачи. Вычислил среднюю выруку с вышки для безубыточной разработки.<br>
Далее создал функцию для расчета прибыли региона.
Применил bootstrap для вычисления распределения прибылей регионов. Распределение получилось следующее:<pre>
1 регион min = -99141931 max = 908834980 руб.
2 регион min = 78088130 max = 848832698 руб.
3 регион min = -113383455 max = 929484372 руб.</pre>
Посчитал вероятность убытков в регионах:<pre>
1)6.2%
2)0.9%
3)6%</pre>
Рассчитал среднюю прибыль в регионах:<pre>
1 регион 392589148 руб.
2 регион 460824976 руб.
3 регион 420877970 руб.</pre>
По завершению исследования лучшим для разработки оказался 2 регион.